In [3]:
from mesa import Agent, Model
from mesa.time import StagedActivation

In [27]:
## Parameters 
N = 5

In [28]:
class DummyAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.model_state = model
        self.id = id
        self.others = list(range(0,N))
        self.others.remove(self.id)
    
    def choose(self):
        # Implementation of move action
        if self.model_state.whose_turn == self.id:
            print("choose" + str(self.id))
        else:
            pass
    
    def vote(self):
        # Implementation of interaction with other agents
        #print("interact")
        pass
    
    def act(self):
        # Update internal state
        #print("update")
        pass


class MyModel(Model):
    def __init__(self, N=N):
        
        super().__init__()

        self.num_agents = N

        self.schedule = StagedActivation(self, stage_list=["choose", "vote", "act"])
        self.round = 0 
        self.whose_turn = self.round % N
        
        # Change this up a little
        for i in range(self.num_agents):  # Example: creating 10 agents
            agent = DummyAgent(i, self)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.round += 1
        self.whose_turn = self.round % N


In [30]:
model = MyModel(N)

for i in range(10):
    model.step()


choose0
choose1
choose2
choose3
choose4
choose0
choose1
choose2
choose3
choose4


In [43]:
x = list(range(0,10))
y = x.remove(5)

In [45]:
x

[0, 1, 2, 3, 4, 6, 7, 8, 9]